In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import math
from mpl_toolkits.mplot3d import Axes3D
import random
import matplotlib.colors as mcolors
import seaborn as sns
from typing import Tuple, List

# preprocessing

In [ ]:
rainfall_data= pd.read_csv('input.csv')
rainfall_data.dropna(inplace=True)
rainfall_data.head(14)

In [ ]:
fig, axs = plt.subplots(12, 2, figsize=(15, 90))
for hour in range(1,24):
    formatted_hour = str(hour).zfill(2)
    i= int((hour-1)/2)
    j= int((hour-1)%2)
    axs[i, j].scatter(rainfall_data['TOTRF'],rainfall_data['HRF'+formatted_hour])
    axs[i, j].set_title('HRF'+formatted_hour)
    hour= hour+1
plt.subplots_adjust(hspace=0.5,wspace=0.3) 
plt.tight_layout()
plt.show()   

# clustering algorithym

In [ ]:
def do_clustering(data_frame,hour,epoch= 3,limit= 30):
    if hour== 25:
        scaler = MinMaxScaler()
        a=scaler.fit_transform(data_frame.iloc[:,[-1]])

        km=KMeans(n_clusters=1,n_init=10, init='k-means++')
        km.fit(a)
        x_first= km.inertia_
        old_x_first= 0
    
        km=KMeans(n_clusters=limit,n_init=10, init='k-means++')
        km.fit(a)
        x_last= km.inertia_
        old_x_last= 0
    
        k= 1
        flag= 1
        array= np.zeros((epoch,2),dtype= float)
        for i in range(epoch) :
            old_x_first= x_first
            old_x_last = x_last
            k = k+1
            km=KMeans(n_clusters=k,n_init=10, init='k-means++')
            km.fit(a)
            x_first= km.inertia_
            km=KMeans(n_clusters=limit-(k-1),n_init=10, init='k-means++')
            km.fit(a)
            x_last= km.inertia_          
            nr        = ((x_last-x_first)+(x_first-old_x_first)*k-(x_last-old_x_last)*(limit-(k-1)))
            dr        = ((x_first-old_x_first)-(x_last-old_x_last))
            array[i,0]= nr/dr
            array[i,1]= (x_first-old_x_first)*((nr/dr)-k)+x_first
        
        reg= linear_model.LinearRegression()    
        reg.fit(array[:, 0].reshape(-1, 1), array[:, 1])
    
        min_cluster= 1
        k= (min_cluster+limit)/2
        k= int(k)
        while min_cluster<limit:
            km=KMeans(n_clusters=k,n_init=10, init='k-means++')
            km.fit(a)
            if reg.predict(np.array([[k]]))>km.inertia_:
                limit= k-1
            if reg.predict(np.array([[k]]))<km.inertia_:
                min_cluster= k+1
            else:
                index= k
            k= int((min_cluster+limit)/2)
        index= k
        
        km = KMeans(n_clusters=index,n_init=10, init='k-means++')
        s  = km.fit_predict(a)
        q  = km.cluster_centers_
    
        data_frame['cluster']=s
        data_frame['TOTRF']=q[data_frame['cluster'],0]
        data_frame.drop(columns=['cluster'],inplace= True)
        data_frame['TOTRF']= scaler.inverse_transform(data_frame.iloc[:,[-1]])[:,0]
        
        return data_frame
    
    #plt.scatter(data_frame.iloc[:,-2],data_frame.iloc[:,-1])
    scaler = MinMaxScaler()
    a=scaler.fit_transform(data_frame.iloc[:,[-1,hour-1]])
    
    km=KMeans(n_clusters=1,n_init=10, init='k-means++')
    km.fit(a)
    x_first= km.inertia_
    old_x_first= 0
    
    km=KMeans(n_clusters=limit,n_init=10, init='k-means++')
    km.fit(a)
    x_last= km.inertia_
    old_x_last= 0
    
    k= 1
    flag= 1
    array= np.zeros((epoch,2),dtype= float)
    for i in range(epoch) :
        old_x_first= x_first
        old_x_last = x_last
        k = k+1
        km=KMeans(n_clusters=k,n_init=10, init='k-means++')
        km.fit(a)
        x_first= km.inertia_
        km=KMeans(n_clusters=limit-(k-1),n_init=10, init='k-means++')
        km.fit(a)
        x_last= km.inertia_          
        nr        = ((x_last-x_first)+(x_first-old_x_first)*k-(x_last-old_x_last)*(limit-(k-1)))
        dr        = ((x_first-old_x_first)-(x_last-old_x_last))
        array[i,0]= nr/dr
        array[i,1]= (x_first-old_x_first)*((nr/dr)-k)+x_first
        
    reg= linear_model.LinearRegression()    
    reg.fit(array[:, 0].reshape(-1, 1), array[:, 1])
    
    min_cluster= 1
    k= (min_cluster+limit)/2
    k= int(k)
    while min_cluster<limit:
        km=KMeans(n_clusters=k,n_init=10, init='k-means++')
        km.fit(a)
        if reg.predict(np.array([[k]]))>km.inertia_:
            limit= k-1
        if reg.predict(np.array([[k]]))<km.inertia_:
            min_cluster= k+1
        else:
            index= k    
        k= int((min_cluster+limit)/2)
    index= k
    
    print('index for hour',hour,'is',index)
    km = KMeans(n_clusters=index,n_init=10, init='k-means++')
    s  = km.fit_predict(a)
    q  = km.cluster_centers_
    data_frame['cluster']=s
    formatted_hour = str(hour).zfill(2)
    data_frame['HRF'+formatted_hour]=q[data_frame['cluster'],1]
    
    data_frame.drop(columns=['cluster'],inplace= True)
    data_frame['HRF'+formatted_hour]= scaler.inverse_transform(data_frame.iloc[:,[-1,hour-1]])[:,1]
                                                                                  
    i= int((hour-1)/2)
    j= int((hour-1)%2)
    axs[i, j].scatter(data_frame['TOTRF'],data_frame['HRF'+formatted_hour],c=s, cmap='tab10')
    axs[i, j].set_title('HRF'+formatted_hour)
    
    return do_clustering(data_frame,hour+1)

In [ ]:
df = rainfall_data.drop(columns=['INDEX', 'YEAR', 'MN', 'DT', 'DAY', 'TOT_Cal'])
df = df[df['TOTRF'] != 0]
fig, axs = plt.subplots(12,2, figsize=(15,90))
result = do_clustering(df,1)
plt.subplots_adjust(hspace=0.5,wspace=0.3) 
plt.tight_layout()
plt.show()            

In [ ]:
result # THE CLUSTER SPACE

# the quarry algorithym

In [ ]:
if 'Season' in rainfall_data.columns:
    rainfall_data.drop(columns=['Season'], inplace=True)
    result.drop(columns=['Season'],inplace=True)

In [ ]:
season_mapping = {
    1: 1, 2: 1, 3: 2,
    4: 2, 5: 2, 6: 3,
    7: 3, 8: 3, 9: 3,
    10: 4, 11: 4, 12: 4
}
rainfall_data['Season'] = rainfall_data['MN'].map(season_mapping)
a= rainfall_data[rainfall_data['TOTRF']!=0]['Season']
result['Season']=a
result.reset_index(drop=True,inplace=True)

trf= float(input('Enter the total rainfall: '))
season= float(input('Enter the season: '))

b = (result['TOTRF'] - trf) ** 2
array = result.iloc[b[b == b.min()].index]
array = array[array['Season']==season]
array.reset_index(drop=True, inplace=True)

fig, axs = plt.subplots(int((len(array)/5)+1),5, figsize=(20,4*int((len(array)/5)+1)))
for index,row in array.iterrows():
    i= int(index/5)
    j= int(index%5)
    axs[i,j].plot(range(len(row.values)-2),row.values[:-2])
    cumulative= row.values[:-2].cumsum()
    axs[i,j].plot(range(len(row.values)-2),cumulative)
    axs[i, j].set_title('Index:'+str(index))
plt.subplots_adjust(hspace=0.5,wspace=0.3) 
plt.tight_layout()
plt.show()

In [ ]:
array # THE QUARRY DATASET

# bias correction

In [ ]:
trf

In [ ]:
def mse(array: np.ndarray, value: float) -> float:
    return (np.sum(array)- value)**2

def seasonal_line(array: np.ndarray, reference: pd.DataFrame) -> Tuple[float, np.ndarray]:
    a = reference.values
    b = np.tile(array, (a.shape[0], 1))
    dot_products = np.sum(a * b, axis=1)
    norms_a = np.linalg.norm(a, axis=1)
    norms_b = np.linalg.norm(b, axis=1)
    cosine_similarities = dot_products / (norms_a * norms_b)
    cosine_similarities = np.nan_to_num(cosine_similarities, nan=0.0)
    fs = np.max(cosine_similarities)
    p = a[np.argmax(cosine_similarities)]
    p_mod = norms_a[np.argmax(cosine_similarities)]
    v_mod = norms_b[np.argmax(cosine_similarities)]
    del_fs = (p / p_mod * v_mod) - (fs * array)
    return fs, del_fs

def forward_pass(input_: np.ndarray, weights: np.ndarray, reference: pd.DataFrame, trf: float) -> Tuple[np.ndarray, float, np.ndarray, np.ndarray, float]:
    op_1 = input_ * weights
    fs, del_fs = seasonal_line(op_1, reference)
    op_2 = np.abs(op_1 * fs)
    error = mse(op_2, trf)
    return op_1, fs, del_fs, op_2, error

def backward_pass(input_: np.ndarray, op_1: np.ndarray, fs: float, del_fs: np.ndarray, op_2: np.ndarray, error: float) -> np.ndarray:
    del_wt = (2 * np.sqrt(error) * (fs + (np.sum(op_1) * del_fs))) * input_
    return del_wt

def parameter_upgrade(weights: np.ndarray, del_wt: np.ndarray, op_2: np.ndarray, trf: float, alpha: float) -> np.ndarray:
    if (np.sum(op_2) - trf) > 0:
        weights = weights - alpha * del_wt
    else:
        weights = weights + alpha * del_wt
    return weights

def Bias_correction(input_: np.ndarray, reference: pd.DataFrame, trf: float, epochs: int = 50, alpha: float = 0.08) -> np.ndarray:
    weights = np.ones_like(input_)
    error_list: List[float] = []
    
    for i in range(epochs):
        op_1, fs, del_fs, op_2, error = forward_pass(input_, weights, reference, trf)
        del_wt = backward_pass(input_, op_1, fs, del_fs, op_2, error)
        weights = parameter_upgrade(weights, del_wt, op_2, trf, alpha)
        error_list.append(error)
    
    plt.plot(range(len(error_list)), error_list, label="Error")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend()
    plt.show()
    
    return op_2

In [ ]:
index= int(input("Enter the index: "))

df= array.drop(columns=['TOTRF','Season'])
df['TOTRF']= np.sum(df.values[:,:],axis=1)
reference = df.copy(deep= True)

reference['TOTRF']= np.sum(reference.values[:,:],axis=1)
scalar    = reference.max().max()
reference = reference/scalar
scaled_trf= trf/scalar
reference.drop(columns=['TOTRF'],inplace=True)

output= Bias_correction(reference.iloc[index,:].values,reference,scaled_trf)
output= np.append(output,np.sum(output))
output= output*(trf/output[-1])

plt.plot(range(len(df.iloc[index,:].values)), df.iloc[index,:].values, label='Input series')
plt.plot(range(len(output)),output,label='Output series')
plt.xlabel("Hour")
plt.ylabel("Rainfall")
plt.legend()
plt.show()

In [ ]:
print(reference.iloc[index,:].values) # the input of bias correction

In [ ]:
print(output) # the output of bias correction